In [33]:
from collections import Counter
import pandas as pd
import numpy as np
import xgboost as xgb
pd.set_option('display.max_colwidth', -1)

In [71]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from utils import UtilsKy

In [77]:
# kyw3
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [90]:
# ky9
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

In [91]:
COL_FACTORS = [ 'amount', 'bank_currency', 'hour', 'day_of_week', 
               'bin', 'longitude', 'latitude', 'phone_2_norm']

In [92]:
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [93]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [94]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [96]:
replace_val = -9999
train.fillna(replace_val)
test.fillna(replace_val)
replace_val

-9999

In [97]:
train = train.values
test = test.values
label = db_teach.status

In [98]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [99]:
result_df_amount = np.nan
weight = analyzer_prediction.get_xgb_weight()

In [100]:
max_depths =[3]
nrounds = [90]
etas = [0.2]

In [101]:
for nround in nrounds:
    for eta in etas:
        for max_depth in max_depths:
            config = {'max_depth':max_depth, 'learning_rate':eta
                      ,'n_estimators':nround }
            model = xgb.XGBClassifier(**config)
            
            model.fit(train, label, eval_metric = 'auc', sample_weight=weight)
             
            test_pred = model.predict_proba(test)
            db_test["probability"] = test_pred[:, 1]
            description = str(config) + 'wl'
            description = '-' . join([str(elem) for elem in (max_depth, eta, nround)])           
            result_df_amount = analyzer_prediction.get_table_prediction()

In [102]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ]
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)

stat_best = stat_best.sort_values(by="rating", ascending=False)
stat_best = stat_best.sort_values(by="p_5", ascending=False)

In [103]:
stat_best

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,,5.03,8.7,12.39,15.69,18.86,21.53,24.49,31.76,52.3,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
